<a href="https://colab.research.google.com/github/AriannaBi/SimpleFCNN/blob/main/SimpleFCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import csv
import torch

Function to read the name of the files

In [58]:
def type_file(name):
    name_file = name.split('/')[3][:3]
    # print(name_file)
    if name_file == 'EDA':
        return 0, 0
    elif name_file == 'BVP':
        return '250ms', 1
    elif name_file == 'ACC':
        return '250ms', 2
    elif name_file == 'TEM':
        return 0, 5
    return -1

Leggo tutti i file per poter creare il tensor con le dimensioni adatte. 
Per ogni file riempio il tensor.

In [79]:
def create_tensor(files):
    leng = 0
    for name_file in files:
        table = pd.read_parquet(name_file, engine='pyarrow')
        df = pd.DataFrame(table)
        timestamp_col = pd.to_datetime(df['timestamp'], unit='s')
        df['timestamp'] = timestamp_col
        arr_type = type_file(name_file)
        if arr_type == -1:
            print("Error in the file name")
            return -1
        if arr_type[0] != 0:
            df = df.resample(arr_type[0], on='timestamp').mean()
            df = df.reset_index()
        leng += df.shape[0]//2400 + int(df.shape[0] % 2400 > 0)
        tensor = np.zeros((6, leng, 2400),dtype=np.float64)
    return tensor


In [80]:
def read_data_locally(files):
    tensor = create_tensor(files)

    # READ ALL THE FILES AND FILL THE TENSOR IN THE CORRESPONDING PLACE
    # array of indices of row to fill
    # EDA, BVP, ACCX, ACCY, ACCZ, TEMP
    arr_idx = [0, 0, 0, 0, 0, 0]
    for name_file in files:
      # DataFrame
      table = pd.read_parquet(name_file, engine='pyarrow')
      # creating DataFrame
      df = pd.DataFrame(table)
      # converting timestamp
      timestamp_col = pd.to_datetime(df['timestamp'], unit='s')
      df['timestamp'] = timestamp_col
      # get if file it's among EDA, BVP, ACC, ST, otherwise return error
      arr_type = type_file(name_file)
      if arr_type == -1:
          print("Error in the file name")
          return -1

      # RESAMPLE the tensor
      # 0 means it's already 4Hz, otherwise resample with '250ms'
      if arr_type[0] != 0:
          df = df.resample(arr_type[0], on='timestamp').mean()
          df = df.reset_index()



      # FILL the tensor
      # number_rows = df.shape[0]//2400 + int(df.shape[0] % 2400 > 0)
      number_rows = df.shape[0]//2400
      print(number_rows, df.shape[0])
      # if not ACC, fill the #st matrix
      if arr_type[1] != 2:
          position_element_in_column = 0
          row_to_fill = arr_idx[arr_type[1]]

          for j in range(row_to_fill, number_rows):
            # everytime it enter the loop add one row
              arr_idx[arr_type[1]] += 1
              for k in range(2400):
                  tensor[arr_type[1]][j][k] = df['value'][position_element_in_column]
                  position_element_in_column += 1
      # otherwise fill ACC_X, ACC_Y, ACC_Z
      elif arr_type[1] == 2:
          list_axis_ACC = ['X', 'Y', 'Z']
          for idx, axis in enumerate(list_axis_ACC):
              index = 2 + idx
              row_to_fill = arr_idx[index]
              position_element_in_column = 0
              for j in range(row_to_fill, number_rows):
                  for k in range(2400):
                      tensor[index][j][k] = df[axis][position_element_in_column]
                      position_element_in_column += 1
          
    print(tensor)
    # for elem in tensor[1]:
    #     print(elem)

    # for i in range(0, len(tensor[1])):
    #   for j in range(0, len(tensor[1][i])):
    #     print(tensor[1][i][j])

      
list_files = ['./drive/MyDrive/ACC_22.parquet', './drive/MyDrive/BVP_41.parquet', './drive/MyDrive/BVP_26.parquet', './drive/MyDrive/TEMP_25.parquet']
read_data_locally(list_files)


126 304393
52 124800
83 199372
53 129024
[[[ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0